<a href="https://colab.research.google.com/github/GTStinson1990/CE397-Project1/blob/main/CE397_Midterm_Project_GTS_Final_R1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CE 397: Scientific Machine Learning**
**Midterm Project**  
**G. Taylor Stinson, P.E., M.S.**  
**October 27, 2025**

**Project Overview:**

This notebook reproduces and evaluates the baseline slope stability artifical neural network (ANN) from Pei & Qiu (2024). The goal is to compare different activation functions and monotonicity constraints on the 168-case slope dataset (Hoang & Pham, 2016). Accuracy, precision, recall, F1, AUC, and monotinicity index (MI), following the evaluation setup in the paper.   

In [4]:
# This must output the file name. If it shows "No such file or directory," the name is wrong.
FILE_PATH = "https://raw.githubusercontent.com/GTStinson1990/CE397-Project1/main/slope_stability_168.csv"

In [5]:
%%writefile data_loader.py
import numpy as np
import pandas as pd
import torch

FEATURE_NAMES = ['gamma', 'c', 'phi', 'beta', 'H', 'ru']
MONOTONIC_INDICATORS = [1, 1, 1, -1, -1, -1]  # +γ,+c,+φ ; −β,−H,−ru

FILE_PATH = "https://raw.githubusercontent.com/GTStinson1990/CE397-Project1/main/slope_stability_168.csv"  # your uploaded file
COLUMNS_TO_LOAD = ['X1','X2','X3','X4','X5','X6','Y']
COLUMN_RENAMES = {
    'X1':'gamma','X2':'c','X3':'phi','X4':'beta','X5':'H','X6':'ru','Y':'Status'
}

def main():
    df = pd.read_csv(FILE_PATH, usecols=COLUMNS_TO_LOAD).rename(columns=COLUMN_RENAMES)

    # Label mapping if needed
    if df['Status'].dtype == object:
        df['Status'] = df['Status'].map({'Stable':1,'Failed':0})
    df['Status'] = df['Status'].astype(int)

    # Filter: c < 200 kPa (paper uses 165 samples)
    before = len(df)
    df = df[df['c'] < 200.0].reset_index(drop=True)
    after = len(df)

    X_raw = df[FEATURE_NAMES].to_numpy(dtype=np.float32)
    y = df['Status'].to_numpy(dtype=np.int64)

    mins = X_raw.min(axis=0)
    maxs = X_raw.max(axis=0)

    torch.save({
        'X_raw': X_raw,
        'y': y,
        'feature_names': FEATURE_NAMES,
        'monotonic_indicators': MONOTONIC_INDICATORS,
        'mins': mins.astype(np.float32),
        'maxs': maxs.astype(np.float32),
    }, 'geotech_data_full.pt')

    n1 = int((y==1).sum())
    n0 = int((y==0).sum())
    print(f"Loaded: {FILE_PATH}  (rows before filter={before}, after={after})")
    print(f"Saved RAW dataset geotech_data_full.pt  X={X_raw.shape}  y={y.shape}  (Stable={n1}, Failed={n0})")
    print(f"Features: {FEATURE_NAMES}")

if __name__ == '__main__':
    main()

Overwriting data_loader.py


In [6]:
!python data_loader.py

Loaded: https://raw.githubusercontent.com/GTStinson1990/CE397-Project1/main/slope_stability_168.csv  (rows before filter=168, after=165)
Saved RAW dataset geotech_data_full.pt  X=(165, 6)  y=(165,)  (Stable=83, Failed=82)
Features: ['gamma', 'c', 'phi', 'beta', 'H', 'ru']


In [7]:
import os, random, numpy as np, pandas as pd, torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

DATA_PT = "geotech_data_full.pt"
FOLDS_PT = "cv_folds.pt"

# ---------------- Reproducibility ----------------
def set_seed(seed: int = 42, deterministic: bool = True):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.use_deterministic_algorithms(True)
        os.environ.setdefault("CUBLAS_WORKSPACE_CONFIG", ":16:8")

# ---------------- Model & penalties ----------------
import torch.nn.functional as F
import torch.autograd as autograd

def monotonic_loss(model, X, mono_inds, loss_type='L2'):
    X = X.detach().clone().requires_grad_(True)
    logits = model(X)              # [B,2]
    target = logits[:, 1]          # logit for "stable"
    grad = autograd.grad(
        outputs=target, inputs=X,
        grad_outputs=torch.ones_like(target),
        create_graph=True, retain_graph=True
    )[0]                           # [B,F]
    mono = torch.as_tensor(mono_inds, dtype=torch.float32, device=X.device)
    viol = F.relu(-grad * mono)
    return (viol**2).mean() if loss_type.upper() == 'L2' else viol.mean()

def get_activation(name: str) -> nn.Module:
    name = (name or "tanh").lower()
    if name == "tanh": return nn.Tanh()
    if name == "relu": return nn.ReLU()
    if name in {"leaky_relu","lrelu","leaky"}: return nn.LeakyReLU(0.01)
    if name == "gelu": return nn.GELU()
    raise ValueError(f"Unsupported activation: {name}")

class MonotonicANN(nn.Module):
    """
    constraint_mode: "none" | "soft" | "hard_global"
      - none: baseline
      - soft: add monotonic_loss to CE in training loop
      - hard_global:
          * flip decreasing features by -1 on input
          * clamp ALL Linear weights >= 0 after each optimizer.step()
          * use monotone activations
    """
    def __init__(self, input_size=6, hidden_size=16, num_hidden=3, output_size=2,
                 activation="tanh", constraint_mode="none",
                 monotonic_indicators=(1,1,1,-1,-1,-1)):
        super().__init__()
        self.constraint_mode = constraint_mode.lower()
        self.mono = torch.tensor(monotonic_indicators, dtype=torch.float32)
        act = get_activation(activation)
        layers, in_f = [], input_size
        for _ in range(num_hidden):
            layers += [nn.Linear(in_f, hidden_size), act.__class__()]
            in_f = hidden_size
        layers += [nn.Linear(in_f, output_size)]
        self.net = nn.Sequential(*layers)
        self.register_buffer("_sign_vec", torch.sign(self.mono))

    def forward(self, x):
        if self.constraint_mode == "hard_global":
            x = x * self._sign_vec
        return self.net(x)  # logits

    @torch.no_grad()
    def project_nonneg_(self, first_layer_only=False):
        for li, m in enumerate(self.net):
            if isinstance(m, nn.Linear):
                if first_layer_only and li != 0: continue
                m.weight.clamp_(min=0.0)

    def apply_hard_constraints_after_step(self):
        if self.constraint_mode == "hard_global":
            self.project_nonneg_(first_layer_only=False)

# ---------------- MI (Algorithm 1: logit, no normalization) ----------------
def estimate_mi_algorithm1(model, X_raw_eval, train_mean_raw, train_std_raw, mono_inds, use_logit=True):
    model.eval()
    X_raw_eval = X_raw_eval.astype(np.float32)
    n, d = X_raw_eval.shape
    assert len(mono_inds) == d

    def model_score(x_scaled_np):
        with torch.no_grad():
            logits = model(torch.tensor(x_scaled_np, dtype=torch.float32))
            if use_logit:
                return logits[:, 1].cpu().numpy()
            probs = torch.softmax(logits, dim=1)[:, 1]
            return probs.cpu().numpy()

    MI_feats = []
    for j, sign in enumerate(mono_inds):
        X_aug = np.tile(train_mean_raw, (n, 1)).astype(np.float32)
        X_aug[:, j] = np.sort(X_raw_eval[:, j])
        X_in = (X_aug - train_mean_raw) / train_std_raw
        f = model_score(X_in)
        df = np.diff(f)
        viol = np.maximum(0.0, -df) if sign > 0 else np.maximum(0.0, +df)
        MI_j = float(viol.sum() / max(len(df), 1))
        MI_feats.append(MI_j)
    return float(np.sum(MI_feats)), MI_feats

# ---------------- Scaling & folds ----------------
def fit_scaler(X_train_raw):
    mean = X_train_raw.mean(axis=0).astype(np.float32)
    std = X_train_raw.std(axis=0, ddof=0).astype(np.float32)
    std[std < 1e-8] = 1.0
    return mean, std

def apply_scaler(X_raw, mean, std):
    return (X_raw - mean) / std

def make_or_load_folds(y, n_splits=5, reps=5, seed=42):
    if os.path.exists(FOLDS_PT):
        return torch.load(FOLDS_PT)["folds"]
    folds = []
    for r in range(reps):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed + r)
        rep = [{"train": tr, "test": te} for tr, te in skf.split(np.zeros_like(y), y)]
        folds.append(rep)
    torch.save({"folds": folds, "seed": seed, "n_splits": n_splits, "reps": reps}, FOLDS_PT)
    return folds

# ---------------- Train/eval one model ----------------
def train_one(Xtr_raw, ytr, Xte_raw, yte, mono_inds,
              activation="tanh", constraint_mode="none",
              k_mono=0.1, epochs=114, batch_size=16, lr=0.005, seed=42):
    set_seed(seed)

    mean_raw, std_raw = fit_scaler(Xtr_raw)
    Xtr = apply_scaler(Xtr_raw, mean_raw, std_raw)
    Xte = apply_scaler(Xte_raw, mean_raw, std_raw)

    g = torch.Generator().manual_seed(seed)
    train_loader = DataLoader(
        TensorDataset(torch.tensor(Xtr, dtype=torch.float32),
                      torch.tensor(ytr, dtype=torch.long)),
        batch_size=batch_size, shuffle=True, generator=g, num_workers=0
    )

    model = MonotonicANN(
        input_size=Xtr.shape[1], hidden_size=16, num_hidden=3, output_size=2,
        activation=activation, constraint_mode=constraint_mode,
        monotonic_indicators=mono_inds
    )
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    ce = nn.CrossEntropyLoss()

    for _ in range(epochs):
        model.train()
        for xb, yb in train_loader:
            opt.zero_grad(set_to_none=True)
            logits = model(xb)
            loss = ce(logits, yb)
            if constraint_mode == "soft":
                loss = loss + k_mono * monotonic_loss(model, xb, mono_inds, loss_type="L2")
            loss.backward(); opt.step()
            if constraint_mode == "hard_global":
                model.apply_hard_constraints_after_step()

    model.eval()
    with torch.no_grad():
        probs = torch.softmax(model(torch.tensor(Xte, dtype=torch.float32)), dim=1)[:, 1].cpu().numpy()
    yhat = (probs >= 0.5).astype(int)

    acc = accuracy_score(yte, yhat)
    prec = precision_score(yte, yhat, zero_division=0)
    rec = recall_score(yte, yhat, zero_division=0)
    f1  = f1_score(yte, yhat, zero_division=0)
    try: auc = roc_auc_score(yte, probs)
    except ValueError: auc = float("nan")

    mi, _ = estimate_mi_algorithm1(
        model, X_raw_eval=Xte_raw, train_mean_raw=mean_raw, train_std_raw=std_raw,
        mono_inds=mono_inds, use_logit=True
    )
    return acc, prec, rec, f1, auc, mi

# ---------------- Main: loop activations × constraints, save CSV/TSV ----------------
def main():
    set_seed(42)
    blob = torch.load(DATA_PT, weights_only=False)
    X_raw, y = blob["X_raw"], blob["y"]
    mono_inds = blob["monotonic_indicators"]
    print(f"Dataset: X={X_raw.shape}, y={y.shape}")

    folds = make_or_load_folds(y, n_splits=5, reps=5, seed=42)

    ACTIVATIONS = ["tanh", "relu", "leaky_relu"]      # add "gelu" if you want
    CONSTRAINTS = ["none", "soft", "hard_global"]     # baseline, soft, hard

    all_rows = []
    for act in ACTIVATIONS:
        for cm in CONSTRAINTS:
            print(f"\n=== Activation: {act} | Constraint: {cm} ===")
            per = []
            for r, rep in enumerate(folds):
                for fidx, f in enumerate(rep):
                    seed = 42 + r * 10 + fidx
                    tr, te = f["train"], f["test"]
                    acc, prec, rec, f1, auc, mi = train_one(
                        X_raw[tr], y[tr], X_raw[te], y[te], mono_inds,
                        activation=act, constraint_mode=cm,
                        k_mono=0.1, epochs=114, batch_size=16, lr=0.005, seed=seed
                    )
                    print(f"[Rep {r+1}/5 Fold {fidx+1}/5] "
                          f"Acc={acc:.3f} Prec={prec:.3f} Rec={rec:.3f} F1={f1:.3f} AUC={auc:.3f} MI={mi:.6f}")
                    per.append([acc, prec, rec, f1, auc, mi])

            per = np.array(per)
            mean, std = per.mean(0), per.std(0)
            print("\n=== 5×5 CV mean ± std ===")
            print(f"Accuracy : {mean[0]:.3f} ± {std[0]:.3f}")
            print(f"Precision: {mean[1]:.3f} ± {std[1]:.3f}")
            print(f"Recall   : {mean[2]:.3f} ± {std[2]:.3f}")
            print(f"F1       : {mean[3]:.3f} ± {std[3]:.3f}")
            print(f"AUC      : {mean[4]:.3f} ± {std[4]:.3f}")
            print(f"MI       : {mean[5]:.6f} ± {std[5]:.6f}")

            out_csv = f"per_fold_metrics_{act}_{cm}.csv"
            pd.DataFrame(per, columns=["Accuracy","Precision","Recall","F1","AUC","MI"]).to_csv(out_csv, index=False)
            all_rows.append({
                "Activation": act, "Constraint": cm,
                "Accuracy_mean": mean[0], "Accuracy_std": std[0],
                "Precision_mean": mean[1], "Precision_std": std[1],
                "Recall_mean": mean[2], "Recall_std": std[2],
                "F1_mean": mean[3], "F1_std": std[3],
                "AUC_mean": mean[4], "AUC_std": std[4],
                "MI_mean": mean[5], "MI_std": std[5],
            })

    pd.DataFrame(all_rows).to_csv("baseline_ann_cv_by_activation_constraint.tsv", sep="\t", index=False)
    print("\nSaved: baseline_ann_cv_by_activation_constraint.tsv")
    print("Also saved per-combo CSVs: per_fold_metrics_<activation>_<constraint>.csv")

if __name__ == "__main__":
    main()


Dataset: X=(165, 6), y=(165,)

=== Activation: tanh | Constraint: none ===
[Rep 1/5 Fold 1/5] Acc=0.848 Prec=0.824 Rec=0.875 F1=0.848 AUC=0.952 MI=0.093330
[Rep 1/5 Fold 2/5] Acc=0.848 Prec=0.867 Rec=0.812 F1=0.839 AUC=0.971 MI=0.105442
[Rep 1/5 Fold 3/5] Acc=0.758 Prec=0.737 Rec=0.824 F1=0.778 AUC=0.926 MI=0.058418
[Rep 1/5 Fold 4/5] Acc=0.970 Prec=0.944 Rec=1.000 F1=0.971 AUC=0.978 MI=0.184949
[Rep 1/5 Fold 5/5] Acc=0.879 Prec=0.882 Rec=0.882 F1=0.882 AUC=0.949 MI=0.181962
[Rep 2/5 Fold 1/5] Acc=0.879 Prec=0.800 Rec=1.000 F1=0.889 AUC=0.879 MI=0.083430
[Rep 2/5 Fold 2/5] Acc=0.879 Prec=0.833 Rec=0.938 F1=0.882 AUC=0.941 MI=0.101213
[Rep 2/5 Fold 3/5] Acc=0.879 Prec=0.882 Rec=0.882 F1=0.882 AUC=0.890 MI=0.344096
[Rep 2/5 Fold 4/5] Acc=0.909 Prec=0.938 Rec=0.882 F1=0.909 AUC=0.956 MI=0.056306
[Rep 2/5 Fold 5/5] Acc=0.970 Prec=1.000 Rec=0.941 F1=0.970 AUC=0.989 MI=0.358745
[Rep 3/5 Fold 1/5] Acc=0.848 Prec=0.789 Rec=0.938 F1=0.857 AUC=0.915 MI=0.110762
[Rep 3/5 Fold 2/5] Acc=0.909 Prec=